#### Ridge for 'score' prediction

In [104]:
import pandas as pd
df = pd.read_csv("./cleaned_data.csv")

from sklearn.linear_model import Ridge

from sklearn.metrics import classification_report
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import numpy as np

In [105]:
X, y = df.drop(['name', 'year', 'score'], axis = 1), df['score']

X['rating'] = X['rating'].astype('category')
X['genre'] = X['genre'].astype('category')
X['director'] = X['director'].astype('category')
X['writer'] = X['writer'].astype('category')
X['star'] = X['star'].astype('category')
X['country'] = X['country'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [108]:
num = list(X_train.select_dtypes('number').columns)
cat = list(X_train.select_dtypes('category').columns)

num_scaled = X_train[num]
for i in num:
    num_scaled[i] = num_scaled[i].apply(lambda x: (x - num_scaled[i].min())/(num_scaled[i].max() - num_scaled[i].min()))
    
encoder = OneHotEncoder().fit(X[cat])
cat_encoded = encoder.transform(X_train[cat]).toarray()

X_train = np.concatenate([num_scaled, cat_encoded], axis=1)

<ipython-input-108-fe0668a5d703>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_scaled[i] = num_scaled[i].apply(lambda x: (x - num_scaled[i].min())/(num_scaled[i].max() - num_scaled[i].min()))


In [109]:
model_ridge = Ridge()
alpha_grid = np.linspace(0.0001, 50, 50)
model_combined_optimal = GridSearchCV(model_ridge, 
                             param_grid = {'alpha': np.linspace(0.0001, 1, 50)})
model_combined_optimal.fit(X_train,y_train)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': array([1.00000000e-04, 2.05061224e-02, 4.09122449e-02, 6.13183673e-02,
       8.17244898e-02, 1.02130612e-01, 1.22536735e-01, 1.42942857e-01,
       1.63348980e-01, 1.83755102e-01, 2.04161224e-01, 2.24567347e-01,
       2.44973469e-01, 2.65379592e-01, 2.85785714e-01, 3.06191837e-01,
       3.26597959e-01, 3.47004082e-01, 3.67410204e-01, 3.87816...
       4.89846939e-01, 5.10253061e-01, 5.30659184e-01, 5.51065306e-01,
       5.71471429e-01, 5.91877551e-01, 6.12283673e-01, 6.32689796e-01,
       6.53095918e-01, 6.73502041e-01, 6.93908163e-01, 7.14314286e-01,
       7.34720408e-01, 7.55126531e-01, 7.75532653e-01, 7.95938776e-01,
       8.16344898e-01, 8.36751020e-01, 8.57157143e-01, 8.77563265e-01,
       8.97969388e-01, 9.18375510e-01, 9.38781633e-01, 9.59187755e-01,
       9.79593878e-01, 1.00000000e+00])})

In [110]:
model_combined_optimal.best_params_

{'alpha': 1.0}

In [111]:
model_combined_optimal.score(X_train,y_train)

0.45253012120898506

In [112]:
num = list(X_test.select_dtypes('number').columns)
cat = list(X_test.select_dtypes('category').columns)

num_scaled = X_test[num]
for i in num:
    num_scaled[i] = num_scaled[i].apply(lambda x: (x - num_scaled[i].min())/(num_scaled[i].max() - num_scaled[i].min()))

cat_encoded = encoder.transform(X_test[cat]).toarray()

X_test = np.concatenate([num_scaled, cat_encoded], axis=1)

<ipython-input-112-58db4fccd799>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_scaled[i] = num_scaled[i].apply(lambda x: (x - num_scaled[i].min())/(num_scaled[i].max() - num_scaled[i].min()))


In [113]:
model_combined_optimal.score(X_test,y_test)

0.40271980311227584